In [1]:
from IPython.display import Image, display, Markdown
import os
import autogen
from autogen.coding import LocalCommandLineCodeExecutor

# When using a single openai endpoint, you can use the following:
config_list = [{"model": "gpt-4o", "api_key": os.getenv("OPENAI_API_KEY")}]


# config_list = [
#     {
#         "model": "gemini-1.5-pro",
#         "api_key": os.getenv("GEMINI_API_KEY"),
#         "api_type": "google"
#     }
# ]

In [4]:
llm_config = {
    "cache_seed": 42,  # change the cache_seed for different trials
    "temperature": 0,
    "config_list": config_list,
    "timeout": 120,
}
user_proxy = autogen.UserProxyAgent(
    name="Admin",
    system_message="A human admin. Interact with the planner to discuss the plan. Plan execution needs to be approved by this admin.",
    code_execution_config={
        "last_n_messages": 3,
        "work_dir": "agent",
        "use_docker": False,
    },  
)
engineer = autogen.AssistantAgent(
    name="Engineer",
    llm_config=llm_config,
    system_message="""Engineer. You follow an approved plan. You write python/shell code to solve tasks. Wrap the code in a code block that specifies the script type. The user can't modify your code. So do not suggest incomplete code which requires others to modify. Don't use a code block if it's not intended to be executed by the executor.
Don't include multiple code blocks in one response. Do not ask others to copy and paste the result. Check the execution result returned by the executor.
If the result indicates there is an error, fix the error and output the code again. Suggest the full code instead of partial code or code changes. If the error can't be fixed or if the task is not solved even after the code is executed successfully, analyze the problem, revisit your assumption, collect additional info you need, and think of a different approach to try.
""",
)

researcher = autogen.AssistantAgent(
    name="Researcher",
    llm_config=llm_config,
    system_message="""Research Navigator. You followed an approved plan. You are able to analyze results, identify relevant sources, and summarize findings. You don't generate code."""
)

planner = autogen.AssistantAgent(
    name="Planner",
    system_message="""Planner. Suggest a plan. Revise the plan based on feedback from admin and critic, until admin approval.
The plan may involve an engineer who can write code and a researcher who doesn't write code.
Explain the plan first. Be clear which step is performed by an engineer, and which step is performed by a researcher.
""",
    llm_config=llm_config,
)

executor = autogen.UserProxyAgent(
    name="Executor",
    system_message="Executor. Execute the code written by the engineer and report the result.",
    human_input_mode="NEVER",
    code_execution_config={
        "last_n_messages": 3,
        "work_dir": "agent",
        "use_docker": False,
    }, 
)
critic = autogen.AssistantAgent(
    name="Critic",
    system_message="Critic. Double check plan, claims, code from other agents and provide feedback. Check whether the plan includes adding verifiable info such as source URL.",
    llm_config=llm_config,
)
groupchat = autogen.GroupChat(
    agents=[user_proxy, engineer, researcher, planner, executor, critic], messages=[], max_round=30
)
manager = autogen.GroupChatManager(groupchat=groupchat, llm_config=llm_config)

In [5]:
user_proxy.initiate_chat(
    manager,
        message="""
   Plan a trip to Sapporo, Japan, focus on sightseeing and food.
    """,
)

Admin (to chat_manager):


   Plan a trip to Sapporo, Japan, focus on sightseeing and food.
    

--------------------------------------------------------------------------------

Next speaker: Planner

Planner (to chat_manager):

### Initial Plan for a Trip to Sapporo, Japan

#### Day 1: Arrival and Initial Exploration
- **Morning:**
  - **Engineer:** Book flights and accommodation in central Sapporo.
  - **Researcher:** Research and list down the best-rated hotels and flights.
- **Afternoon:**
  - **Researcher:** Create a list of must-visit places in Sapporo.
  - **Engineer:** Arrange airport transfer to the hotel.
- **Evening:**
  - **Researcher:** Identify a popular local restaurant for dinner.
  - **Engineer:** Make a reservation at the selected restaurant.

#### Day 2: Historical and Cultural Exploration
- **Morning:**
  - **Researcher:** Plan a visit to the Hokkaido Shrine and the Historical Village of Hokkaido.
  - **Engineer:** Arrange transportation to these locations.
- **Af

ChatResult(chat_id=None, chat_history=[{'content': '\n   Plan a trip to Sapporo, Japan, focus on sightseeing and food.\n    ', 'role': 'assistant'}, {'content': '### Initial Plan for a Trip to Sapporo, Japan\n\n#### Day 1: Arrival and Initial Exploration\n- **Morning:**\n  - **Engineer:** Book flights and accommodation in central Sapporo.\n  - **Researcher:** Research and list down the best-rated hotels and flights.\n- **Afternoon:**\n  - **Researcher:** Create a list of must-visit places in Sapporo.\n  - **Engineer:** Arrange airport transfer to the hotel.\n- **Evening:**\n  - **Researcher:** Identify a popular local restaurant for dinner.\n  - **Engineer:** Make a reservation at the selected restaurant.\n\n#### Day 2: Historical and Cultural Exploration\n- **Morning:**\n  - **Researcher:** Plan a visit to the Hokkaido Shrine and the Historical Village of Hokkaido.\n  - **Engineer:** Arrange transportation to these locations.\n- **Afternoon:**\n  - **Researcher:** Find a traditional J

In [8]:
from IPython.display import Markdown

with open('/workspaces/JourneyCraftAI/notebooks/agent/trip_plan_sapporo.md', 'r') as f:
    markdown_text = f.read()

Markdown(markdown_text)  



# Trip Plan to Sapporo, Japan

## Overview
This trip plan to Sapporo, Japan, focuses on sightseeing and food. The itinerary includes visits to historical sites, parks, local restaurants, and shopping districts. The coordinates for each place have been plotted on an interactive map.

## Places and Coordinates

| Order of Visit | Place Name                    | Latitude         | Longitude          | Description                                                                 |
|----------------|-------------------------------|------------------|--------------------|-----------------------------------------------------------------------------|
| 1              | JR Tower Hotel Nikko Sapporo  | 43.06821815      | 141.35238160436768 | Centrally located hotel in Sapporo.                                         |
| 2              | Sapporo Clock Tower           | 43.0625512       | 141.35364549007318 | Historical clock tower and symbol of Sapporo.                               |
| 3              | Sapporo Beer Garden           | 43.0719737       | 141.3682368        | Popular restaurant offering local beer and cuisine.                         |
| 4              | Hokkaido Shrine               | 43.05466585      | 141.30919279140036 | Shinto shrine located in Maruyama Park.                                     |
| 5              | Historical Village of Hokkaido| 43.0460191       | 141.49854487492314 | Open-air museum showcasing historical buildings from Hokkaido.              |
| 6              | Sapporo Concert Hall Kitara   | 43.0443087       | 141.3522851267233  | Concert hall hosting various performances.                                  |
| 7              | Moerenuma Park                | 43.122028549999996 | 141.4285473179036 | Large park designed by Isamu Noguchi, featuring sculptures and playgrounds. |
| 8              | Odori Park                    | 43.05990175      | 141.3475100757782  | Central park in Sapporo, known for its seasonal festivals.                  |
| 9              | Gotsubo                       | 43.0536731       | 141.3543672        | Local izakaya (Japanese pub) offering a variety of dishes.                  |
| 10             | Tanukikoji Shopping Street    | 43.0569761       | 141.35058313470506 | Covered shopping arcade with various shops and restaurants.                 |
| 11             | Sapporo Factory               | 43.065259499999996 | 141.36253759104648 | Shopping complex with shops, restaurants, and entertainment options.        |
| 12             | Ramen Shingen                 | 43.0522632       | 141.3475335        | Renowned ramen shop known for its miso ramen.                               |
| 13             | Jozankei Onsen                | 42.9648301       | 141.1601262        | Hot spring resort area near Sapporo.                                        |

## Interactive Map
The interactive map below shows the locations of the places mentioned in the trip plan. The places are connected in sequence using AntPath.

<iframe src="sapporo_trip_map.html" width="800" height="600"></iframe>


### Notes:
- Places without coordinates have been removed from the table.
- Descriptions provide a brief overview of each place to give context to the visit.

This table provides a clear and organized overview of the trip plan, including the order of visit, place names, coordinates, and descriptions.
